In [1]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pandas as pd

In [2]:
# Disable TensorFlow Caching
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

# Ensure limited GPU memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Batch sizes for testing
BATCH_SIZES = [1, 4, 8, 16, 32, 64, 100, 128]
NUM_RUNS = 10  # Runs per batch size

# Load NASNetMobile model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow function to avoid extra optimizations
@tf.function(experimental_relax_shapes=True)
def run_inference(input_tensor):
    return model(input_tensor)

2025-02-05 15:40:25.877334: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-05 15:40:25.877403: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-05 15:40:25.877415: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-05 15:40:25.877466: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-05 15:40:25.877507: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Load CIFAR-100 dataset
def load_cifar100(batch_size):
    dataset = tfds.load("cifar100", split="train", as_supervised=True)
    dataset = dataset.map(lambda x, y: (tf.image.resize(x, [224, 224]), y))  # Resize to 224x224
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [4]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size, dataset_iter):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Load Dataset
        try:
            input_tensor, _ = next(dataset_iter)  # Get real ImageNet images
        except StopIteration:
            dataset_iter = iter(dataset)  # Restart dataset if exhausted
            input_tensor, _ = next(dataset_iter)
        
        with tf.device(device):
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

            # Manually clear TensorFlow cache after each batch run
            del input_tensor
            tf.keras.backend.clear_session

    return results

## Experiment Logging

In [5]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []

for device in devices:    
    for batch_size in BATCH_SIZES:
        dataset = load_cifar100(batch_size)
        dataset_iter = iter(dataset)
        try:
            tf.profiler.experimental.start(f"logs/Image/NoCache/{device[1:4]}/{batch_size}")
            final_results.extend(benchmark_model(device, batch_size, dataset_iter))
        finally:
            tf.profiler.experimental.stop()

2025-02-05 15:40:28.721414: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 15:40:28.721424: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 1


2025-02-05 15:40:30.355691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-05 15:40:31.606747: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:40:31.738114: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:40:31.739589: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/NoCache/CPU/1/plugins/profile/2025_02_05_15_40_31/C17586.xplane.pb
2025-02-05 15:40:31.794466: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).


Running on /CPU:0 with batch size 4


2025-02-05 15:40:35.418635: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:40:35.562095: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:40:35.562359: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/NoCache/CPU/4/plugins/profile/2025_02_05_15_40_35/C17586.xplane.pb
2025-02-05 15:40:35.620912: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:40:35.623239: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-0

...

2025-02-05 15:44:34.042880: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:44:34.358795: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:44:34.365599: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/NoCache/GPU/64/plugins/profile/2025_02_05_15_44_34/C17586.xplane.pb
2025-02-05 15:44:35.064220: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:44:35.085987: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-


Running on /GPU:0 with batch size 100


2025-02-05 15:45:50.628749: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:45:51.087761: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:45:51.319369: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/NoCache/GPU/100/plugins/profile/2025_02_05_15_45_51/C17586.xplane.pb
2025-02-05 15:45:52.661017: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:45:52.757074: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025


Running on /GPU:0 with batch size 128


2025-02-05 15:48:02.255969: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:48:02.808137: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:48:02.816898: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/NoCache/GPU/128/plugins/profile/2025_02_05_15_48_02/C17586.xplane.pb


## Results

In [6]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.032033
1,/CPU:0,1,0.033857
2,/CPU:0,1,0.033057
3,/CPU:0,1,0.037750
4,/CPU:0,1,0.037185


In [7]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                               
                         min      mean       var        max
device batch_size                                          
/CPU:0 1            0.032033  0.034255  0.000004   0.037750
       4            0.063854  0.067680  0.000013   0.075027
       8            0.094223  0.099022  0.000024   0.109501
       16           0.154989  0.166904  0.000063   0.182872
       32           0.293766  0.328750  0.001877   0.449149
       64           0.778447  1.839392  0.147828   2.166127
       100          2.351557  3.057380  0.768339   5.390888
       128          3.122175  3.520736  0.043990   3.834079
/GPU:0 1            0.085520  0.102317  0.001138   0.171663
       4            0.087268  0.127457  0.006467   0.291039
       8            0.090430  0.106536  0.001976   0.232786
       16           0.089931  0.142024  0.013263   0.440057
       32           0.129961  0.184626  0.013479   0.467252
       64           1.051903  1.396064  0.087050   1.967707
       100          2.217554  5.126023  5.778403   8.062158
       128          3.052357  8.784103  6.623561  11.908275